In [1]:
import pandas as pd
from load_tables import table_loader
import json

In [59]:
def return_time(float):
    d_start = -1262304000
    d_end = 1514764800
    unix_time = (d_end - d_start)*float+d_start
    return pd.to_datetime(unix_time, unit='s').to_period('d').to_timestamp()

In [63]:
class table_loader():
    def __init__(self):
        self.movie_actorDF = pd.read_csv("final_MovieToActor.csv",index_col=0)
        self.actorDF = pd.read_csv("final_Actors.csv", index_col=0)
        self.metaDataDF = pd.read_csv("final_MoviesMetaData_processed.csv", index_col=0)
        self.metaDataDF.loc[:,'release_date'] = pd.to_datetime(self.metaDataDF['release_date'])
        self.movie_to_genreDF = pd.read_csv('final_MovieToGenre.csv', index_col=0)
        self.Mean = 1

    def return_filtered(self, start_float, end_float, max_num=50, genres='All'):

        start_t = return_time(start_float)
        end_t = return_time(end_float)
        time_contrainedMetaFD = self.metaDataDF[(self.metaDataDF['release_date'] > start_t) & (self.metaDataDF['release_date'] < end_t)]
        if genres == 'All':
            pass
        else:
            table = self.movie_to_genreDF[self.movie_to_genreDF.name.isin(genres)][['id','name']].groupby('id').count()
            table = table[table.name==len(genres)]
            table.loc[:,'id'] = table.index
            time_contrainedMetaFD = pd.merge(table,time_contrainedMetaFD)
        start_d = pd.to_datetime(start_t)
        end_d = pd.to_datetime(end_t)
        scoreDF = pd.merge(self.movie_actorDF, time_contrainedMetaFD)[['actor_id', 'final_score']].groupby('actor_id').sum()
        scoreDF = scoreDF / scoreDF.sort_values('final_score', ascending=False).iloc[:max_num].sum()
        time_contrainedMetaFD.loc[:,'relative_position'] = (time_contrainedMetaFD.release_date - start_d) / (end_d - start_d)
        self.time_contrainedMetaFD=time_contrainedMetaFD
        final_table = pd.merge(self.movie_actorDF, time_contrainedMetaFD)[['actor_id', 'relative_position']].groupby(
            'actor_id').mean().join(self.actorDF).join(scoreDF).sort_values('final_score', ascending=False).iloc[:max_num]
        final_table.loc[:,'actor_id'] =  final_table.index
        #columns_table = self.return_revenue_chart(num_columns,self.Mean)
        return final_table.to_json(orient='index')#.to_csv()#, ','.join(columns_table.astype(str))) #.to_json(orient='index')

    def return_revenue_chart(self, num_columns, Mean = 1):
        self.mean = Mean
        if Mean:
            return ','.join(self.time_contrainedMetaFD[self.time_contrainedMetaFD.revenue!=0].groupby(
            pd.cut(self.time_contrainedMetaFD[self.time_contrainedMetaFD.revenue!=0]["relative_position"],
                   np.arange(0, 1.0+1/num_columns, 1/num_columns)))['revenue'].mean().values.astype(str))
        else:
            return ','.join(self.time_contrainedMetaFD[self.time_contrainedMetaFD.revenue != 0].groupby(
                pd.cut(self.time_contrainedMetaFD[self.time_contrainedMetaFD.revenue != 0]["relative_position"],
                       np.arange(0, 1.0 + 1 / num_columns, 1 / num_columns)))['revenue'].sum().values.astype(str))

    def return_actor_network(self,actor_id):
        pd.merge(pd.merge(self.movie_actorDF[self.movie_actorDF.actor_id == actor_id], self.metaDataDF), self.movie_to_genreDF)
        DF = pd.merge(
            pd.merge(pd.merge(self.movie_actorDF[self.movie_actorDF.actor_id == actor_id], self.metaDataDF), self.movie_to_genreDF)[
                ['id', 'genre_id']], self.movie_actorDF)
        DF2 = DF[DF.actor_id != actor_id].drop_duplicates(['id', 'actor_id'])
        self.actorDF.loc[:,'actor_id'] = self.actorDF.index
        DF3 = pd.merge(DF2, self.actorDF)
        DF4 = DF3.sort_values('all_time_final_score', ascending=False).iloc[:50]
        actor_dictionary = self.actorDF[['name', 'gender', 'all_time_final_score','actor_id']].to_dict(orient='index')
        actor_set = set()
        actor_dict_list = [{'name': actor_dictionary[actor_id]['name'], 'gender': actor_dictionary[actor_id]['gender'],
                            'score': actor_dictionary[actor_id]['all_time_final_score'], 'actor_id':actor_dictionary[actor_id]['actor_id']}]
        links_list = []
        skip = 0
        for index, (index_row, row) in enumerate(DF4.iterrows()):

            links_list.append({'source': 0, "target": len(links_list) - skip, 'value': row.genre_id})
            if row.actor_id in actor_set:
                skip += 1
                continue
            #print(row)
            actor_set.add(row.actor_id)
            actor_dict_list.append(
                {'name': actor_dictionary[row.actor_id]['name'], 'gender': actor_dictionary[row.actor_id]['gender'],
                 'score': actor_dictionary[row.actor_id]['all_time_final_score'], 'actor_id':actor_dictionary[row.actor_id]['actor_id']})
        return json.dumps({"actors":actor_dict_list, "movies":links_list})


In [64]:
loader = table_loader()

In [65]:
loader.return_actor_network(2231)

'{"actors": [{"name": "Samuel L. Jackson", "gender": 2, "score": 457.343116678874, "actor_id": 2231}, {"name": "Tom Hanks", "gender": 2, "score": 392.26903862409074, "actor_id": 31}, {"name": "Sandra Bullock", "gender": 1, "score": 390.5348017356965, "actor_id": 18277}, {"name": "Robert De Niro", "gender": 2, "score": 387.9057361089489, "actor_id": 380}, {"name": "Michael Caine", "gender": 2, "score": 374.40415142073664, "actor_id": 3895}, {"name": "Bruce Willis", "gender": 2, "score": 365.42308237389335, "actor_id": 62}, {"name": "Robert Downey Jr.", "gender": 2, "score": 349.86696782807974, "actor_id": 3223}, {"name": "Brad Pitt", "gender": 2, "score": 327.70813154256837, "actor_id": 287}, {"name": "Michael Keaton", "gender": 2, "score": 312.88762986042804, "actor_id": 2232}, {"name": "Harrison Ford", "gender": 2, "score": 297.6883434969454, "actor_id": 3}, {"name": "Christian Bale", "gender": 2, "score": 289.02466121895003, "actor_id": 3894}, {"name": "Gary Oldman", "gender": 2, "sc

In [118]:
movie_actorDF = pd.read_csv("final_MovieToActor.csv",index_col=0)
actorDF = pd.read_csv("final_Actors.csv", index_col=0)
metaDataDF = pd.read_csv("final_MoviesMetaData_processed.csv", index_col=0)
metaDataDF.loc[:,'release_date'] = pd.to_datetime(metaDataDF['release_date'])


In [119]:
actorDF['actor_id'] = actorDF.index
start_t = return_time(0)
end_t = return_time(1)
start_d = pd.to_datetime(start_t)
end_d = pd.to_datetime(end_t)

In [121]:
DF = pd.merge(movie_actorDF[movie_actorDF.actor_id == 2231],metaDataDF)#
DF.loc[:,'relative_position'] = (DF.release_date - start_d) / (end_d - start_d)
#DF.loc[:,'circle_size'] = DF['score']/DF['score'].sum()
DF[['id','title','relative_position','vote_average','score']].to_json(orient='index')

'{"0":{"id":1572,"title":"Die Hard: With a Vengeance","relative_position":0.7429220335,"vote_average":6.9,"score":8.9514199251},"1":{"id":21183,"title":"Fluke","relative_position":0.7433576006,"vote_average":6.6,"score":2.6383615749},"2":{"id":6071,"title":"Kiss of Death","relative_position":0.7420508991,"vote_average":5.7,"score":2.6895813189},"3":{"id":28313,"title":"Losing Isaiah","relative_position":0.7409619812,"vote_average":6.4,"score":3.7855681076},"4":{"id":680,"title":"Pulp Fiction","relative_position":0.7351129363,"vote_average":8.3,"score":114.3271876123},"5":{"id":13815,"title":"Fresh","relative_position":0.7345840334,"vote_average":7.6,"score":2.8284858871},"6":{"id":329,"title":"Jurassic Park","relative_position":0.7209258914,"vote_average":7.6,"score":6.3147878194},"7":{"id":9516,"title":"Menace II Society","relative_position":0.7204281003,"vote_average":7.2,"score":3.3990891974},"8":{"id":41588,"title":"The New Age","relative_position":0.735299608,"vote_average":4.8,"s

In [ ]:
    def return_cloud_points(self,actor_id):
        
    start_t = return_time(0)
    end_t = return_time(1)
    start_d = pd.to_datetime(start_t)
    end_d = pd.to_datetime(end_t)
    DF = pd.merge(movie_actorDF[movie_actorDF.actor_id == actor_id],metaDataDF)#
        DF.loc[:,'relative_position'] = (DF.release_date - start_d) / (end_d - start_d)
        #DF.loc[:,'circle_size'] = DF['score']/DF['score'].sum()
        DF[['id','title','relative_position','vote_average','score']].to_json(orient='index')

In [2]:
df = pd.read_csv("final_Actors.csv")

In [12]:
with open("actorID.json","w") as f:
    f.write(json.dumps(df[['id','name']].to_dict('records')))
    
import json
with open('actorID.json', 'w') as outfile:
    json.dump(json.dumps(df[['id','name']].to_dict('records')), outfile)

In [33]:
dict_ = []
for index, x in df[['id','name']].iterrows():
    dict_.append({"id":x.id,"name":x["name"]})
    

In [42]:
with open('actorID.json', 'w') as outfile:
    json.dump(json.dumps(dict_), outfile)